# Notebook for Splunk Machine Learning Toolkit Container for TensorFlow

This notebook contains an example workflow how to work on custom containerized code that seamlessly interfaces with the Splunk Machine Learning Toolkit (MLTK) Container for TensorFlow. This script contains an example of how to run an entity extraction algorithm over text using the spacy library.

## Stage 0 - import libraries
At stage 0 we define all imports necessary to run our subsequent code depending on various libraries.

In [1]:
# this definition exposes all python module imports that should be available in all subsequent commands
import json
import datetime
import numpy as np
import pandas as pd
import spacy

# global constants
MODEL_DIRECTORY = "/srv/app/model/data/"

In [2]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
print("numpy version: " + np.__version__)
print("pandas version: " + pd.__version__)
print("spacy version: " + spacy.__version__)


numpy version: 1.16.4
pandas version: 0.25.1
spacy version: 2.2.4


In [3]:
import ja_ginza
nlp = ja_ginza.load()

## Stage 1 - get a data sample from Splunk
In Splunk run a search to pipe a prepared dataset into this environment.

| makeresults
| eval text = "アメリカのトランプ大統領は、新型コロナウイルス対策で制限を受けている経済活動について、20日から、テキサス州などで一部再開すると発表した。トランプ大統領「テキサス州とバーモント州では、20日に一部のビジネスを開始するだろうが、引き続き、感染予防としてソーシャルディスタンスを求める」。またトランプ大統領は、モンタナ州では、24日に経済活動の規制を解除し、5月1日からは、オハイオ、ノースダコタ、それにアイダホの各州で、段階的に経済活動再開の準備を進めていることを明らかにした。トランプ大統領が公表した、経済活動を3段階に分けて再開するとしたガイドラインでは、第1段階で、テレワークを継続しながら通勤も可能などとしているが、「安全を優先させるべき」と慎重な声も多くある。"
| makemv text delim="。"
| mvexpand text
| fit MLTKContainer algo=spacy_ner_ja epochs=100 text into app:spacy_ginza_entity_extraction_model

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code

After you run this search your data set sample is available as a csv inside the container to develop your model. The name is taken from the into keyword ("spacy_ginza_entity_extraction_model in the example above) or set to "default" if no into keyword is present. This step is intended to work with a subset of your data to create your custom model.

In [4]:
# this cell is not executed from MLTK and should only be used for staging data into the notebook environment
def stage(name):
    with open("data/"+name+".csv", 'r') as f:
        df = pd.read_csv(f)
    with open("data/"+name+".json", 'r') as f:
        param = json.load(f)
    return df, param

In [5]:
# THIS CELL IS NOT EXPORTED - free notebook cell for testing purposes
df, param = stage("spacy_ginza_entity_extraction_model")
print(df)
print(df.shape)
print(str(param))

                                                text
0  アメリカのトランプ大統領は、新型コロナウイルス対策で制限を受けている経済活動について、20日...
1  トランプ大統領「テキサス州とバーモント州では、20日に一部のビジネスを開始するだろうが、引き...
2  またトランプ大統領は、モンタナ州では、24日に経済活動の規制を解除し、5月1日からは、オハイ...
3  トランプ大統領が公表した、経済活動を3段階に分けて再開するとしたガイドラインでは、第1段階で...
(4, 1)
{'options': {'params': {'mode': 'stage', 'algo': 'spacy_ner_ja', 'epochs': '100'}, 'args': ['text'], 'feature_variables': ['text'], 'model_name': 'spacy_ginza_entity_extraction_model', 'algo_name': 'MLTKContainer', 'mlspl_limits': {'handle_new_cat': 'default', 'max_distinct_cat_values': '100', 'max_distinct_cat_values_for_classifiers': '100', 'max_distinct_cat_values_for_scoring': '100', 'max_fit_time': '600', 'max_inputs': '100000', 'max_memory_usage_mb': '1000', 'max_model_size_mb': '15', 'max_score_time': '600', 'streaming_apply': 'false', 'use_sampling': 'true'}, 'kfold_cv': None}, 'feature_variables': ['text']}


## Stage 2 - create and initialize a model

In [6]:
# initialize the model
# params: data and parameters
# returns the model object which will be used as a reference to call fit, apply and summary subsequently
def init(df,param):
    # Load Japanese tokenizer, tagger, parser, NER and word vectors
    import ja_ginza
    model = ja_ginza.load()
    #model = spacy.load("ja_ginza")
    return model

In [7]:
model = init(df,param)

## Stage 3 - fit the model

Note that for this algorithm the model is pre-trained (the ja_ginza library comes pre-packaged by Ginza) and therefore this stage is a placeholder only

In [8]:
# returns a fit info json object
def fit(model,df,param):
    returns = {}
    
    return returns

## Stage 4 - apply the model

In [9]:
def apply(model,df,param):
    X = df[param['feature_variables']].values.tolist()
    
    returns = list()
    
    for i in range(len(X)):
        doc = model(str(X[i]))
        
        
        entities = ''
    
        # Find named entities, phrases and concepts
        for entity in doc.ents:
            if entities == '':
                entities = entities + entity.text + ':' + entity.label_
            else:
                entities = entities + '|' + entity.text + ':' + entity.label_
        
        returns.append(entities)
    return returns

In [10]:
returns = apply(model,df,param)

## Stage 5 - save the model

In [11]:
# save model to name in expected convention "<algo_name>_<model_name>.h5"
def save(model,name):
    # model will not be saved or reloaded as it is pre-built
    return model

## Stage 6 - load the model

In [12]:
# load model from name in expected convention "<algo_name>_<model_name>.h5"
def load(name):
    # model will not be saved or reloaded as it is pre-built
    return model

## Stage 7 - provide a summary of the model

In [13]:
# return model summary
def summary(model=None):
    returns = {"version": {"spacy": spacy.__version__} }
    if model is not None:
        # Save keras model summary to string:
        s = []
        returns["summary"] = ''.join(s)
    return returns

## End of Stages
All subsequent cells are not tagged and can be used for further freeform code